# Урок 2. Профилирование пользователей. Сегментация аудитории: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)

#### Домашнее задание

1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

#### Ссылки

1. http://www.machinelearning.ru/wiki/images/d/d5/Voron17survey-artm.pdf
2. https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation

## Выполнение

### 
Самостоятельно разобраться с тем, что такое tfidf. Документация: 
- https://scikitlearn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html 
- https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)

### 
Модифицировать код функции `get_user_embedding` таким образом, чтобы считалось не среднее (как в примере `np.mean`), а медиана. 

Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: `roc auc`, `precision/recall/f_score` (для 3 последних - подобрать оптимальный порог с помощью `precision_recall_curve`, как это делалось на уроке)

In [1]:
import pandas as pd

#### Загрузим новости, пользователей пользователей и списки последних прочитанных новостей

In [2]:
news = pd.read_csv("Data/articles.csv")
users = pd.read_csv("Data/users_articles.csv")

In [3]:
display(news.head(3), users.head(3))

,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

#### Получаем векторные представления новостей

In [4]:
"Gensim — библиотека обработки естественного языка предназначения для «Тематического моделирования»."
# !pip install --upgrade gensim

"""
razdel - rule-based system for Russian sentence and word tokenization. See natasha.github.io article for more info.
https://github.com/natasha/razdel"""
# !pip install razdel

"Морфологический анализатор pymorphy2"
# !pip install pymorphy2

'Морфологический анализатор pymorphy2'

In [5]:
#предобработка текстов
from gensim.corpora.dictionary import Dictionary
import re
import numpy as np
from razdel import tokenize  # https://github.com/natasha/razdel
import pymorphy2

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stopword_ru = nltk.corpus.stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [7]:
with open('Data/stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

777

In [13]:
def clean_text(text):
    """
    Очистка текста
    :return: Очищеный текст
    """
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '',
                  text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())

    # tokens = list(tokenize(text))
    # words = [_.text for _ in tokens]
    # words = [w for w in words if w not in stopword_ru]

    # return " ".join(words)
    return text


cache = {}


def lemmatization(text):
    """
    Лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова
    :return: Список отлемматизированых токенов
    """
    # [0]
    if not isinstance(text, str):
        text = str(text)

    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-':  # [2]
            w = w[1:]
        if len(w) > 1:  # [3]
            if w in cache:  # [4]
                words_lem.append(cache[w])
            else:  # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)

    words_lem_without_stopwords = [
        i for i in words_lem if not i in stopword_ru
    ]  # [6]

    return words_lem_without_stopwords

In [9]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), axis=1)

<ipython-input-8-fd48c693ac78>:13: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 52.2 s


In [10]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), axis=1)

Wall time: 5min 52s


#### А теперь в 3 строчки обучим нашу модель

In [11]:
%%time
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Wall time: 20.7 s


Что такое common_dictionary и как он выглядит

In [14]:
print([common_dictionary[i] for i in range(10, 15)])

['взаимодействие', 'власть', 'войти', 'вячеслав', 'генералполковник']


Все просто - это словарь наших слов

Запускаем обучение

In [15]:
%%time
from gensim.models import LdaModel

# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary, passes=5)

Wall time: 4min 42s


In [16]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [17]:
# Create a new corpus, made of previously unseen documents.
# Создайте новый корпус, состоящий из ранее невидимых документов.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]

print(other_texts[2])
lda[unseen_doc]

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(11, 0.25075376),
 (15, 0.038594566),
 (19, 0.10436495),
 (20, 0.47758815),
 (24, 0.111271605)]

In [18]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: сша американский военный китай напомнить сила северный
topic_1: компания который дело год это решение суд
topic_2: озеро гражданство су рисунок круглый лечь конкурентный
topic_3: год проект который новый научный наука работа
topic_4: граница долг министерство российский японский конкурс расчёт
topic_5: рубль млн рост год бюджет доход сумма
topic_6: млрд фонд риск европа двигатель германия год
topic_7: россия российский президент это заявить путин страна
topic_8: пенсия спрос иран сектор добавлять виза индия
topic_9: год человек который ребёнок свой это жизнь
topic_10: год рак суд статья журнал тыс срок
topic_11: это мочь который всё весь говорить наш
topic_12: турция восток турецкий городской разместить свидетель лекарство
topic_13: гражданин квартира россиянин производитель письмо народный фронт
topic_14: год рынок цена уровень исследование эксперт это
topic_15: ракета произойти район находиться данные пострадать результат
topic_16: закон который депутат санкция писать право 

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [19]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(
        zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [20]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.042659,0.0,0.0,0.0,0.0,0.024368,0.035171,0.000000,...,0.0000,0.0,0.0,0.0,0.000000,0.098689,0.0,0.758606,0.0,0.033256
1,4896,0.0,0.166916,0.0,0.0,0.0,0.0,0.000000,0.029122,0.000000,...,0.0000,0.0,0.0,0.0,0.000000,0.781958,0.0,0.000000,0.0,0.000000
2,4897,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0386,0.0,0.0,0.0,0.104366,0.477603,0.0,0.000000,0.0,0.111273
3,4898,0.0,0.155875,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0000,0.0,0.0,0.0,0.000000,0.345854,0.0,0.000000,0.0,0.000000
4,4899,0.0,0.518415,0.0,0.0,0.0,0.0,0.000000,0.102457,0.143141,...,0.0000,0.0,0.0,0.0,0.000000,0.123204,0.0,0.000000,0.0,0.090537


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

In [21]:
# users.head(3)

In [22]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [27]:
# doc_dict[293622]

In [36]:
user_articles_list = users['articles'].iloc[33]
display(user_articles_list, 
        eval(user_articles_list)
       )

'[323329, 321961, 324743, 323186, 324632, 474690]'

[323329, 321961, 324743, 323186, 324632, 474690]

---

In [73]:
def get_user_embedding_mean(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

def get_user_embedding_median(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [138]:
def score_with_user_embedding(func):
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: func(x), 1)])
    user_embeddings.columns = [f'topic_{i}' for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid'] + [f'topic_{i}' for i in range(25)]]

    X = pd.merge(user_embeddings, target, 'left')

    #разделим данные на train/test
    X_train, X_test, y_train, y_test = train_test_split(
    X[[f'topic_{i}' for i in range(25)]], X['churn'], random_state=0)
    
    logreg = LogisticRegression()

    #обучим наш пайплайн
    logreg.fit(X_train, y_train)

    #наши прогнозы для тестовой выборки
    preds = logreg.predict_proba(X_test)[:, 1]

    # Рассчитаем Precision, Recall, F_score, roc_auc_score
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc_score = roc_auc_score(y_test, preds)

    return round(thresholds[ix], 4), round(fscore[ix], 4), round(precision[ix], 4), round(recall[ix], 4), round(roc_auc_score, 4)

In [94]:
target = pd.read_csv("Data/users_churn.csv")

### 
Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: `mean`, `median`, `max`, `idf_mean` по метрикам `roc_auc`, `precision`, `recall`, `f_score`

In [136]:
results = pd.DataFrame(np.array([
    score_with_user_embedding(func=get_user_embedding_mean),
    score_with_user_embedding(func=get_user_embedding_median),
    score_with_user_embedding(func=get_user_embedding_max)
]), columns=['Best Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC score'])

results['func'] = ['mean', 'median', 'max']
results = results.set_index('func')

In [137]:
results

,Best Threshold,F-Score,Precision,Recall,ROC AUC score
func,,,,,
mean,0.3245,0.8531,0.8531,0.8531,0.9855
median,0.2904,0.8442,0.8168,0.8735,0.9854
max,0.3667,0.8853,0.8730,0.8980,0.9880


### 
Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

```python
Судя по цифрам, самый эффективный способ - через max. 
Возможные причины - разброс в интесесах пользователя становится меньше, интересы как-бы более узкие, но более точные.
```